In [1]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk
import scipy.constants as cs

![Gilbert cell Topology](img/topology_Gilbert_Cell.png)

![diff_pair](img/diff_pair.png)

#### Technology data

In [2]:
n = lk('./nfet_03v3.mat')
p = lk('./pfet_03v3.mat')

#### Specifications

In [3]:
f_rf = 2.4 * cs.giga
f_lo = 2.5 * cs.giga
NF = 9 # [dB], baseband integrated from 1K to 100MHz
VDD = 3.3 # [V]
conversion_gain = 15 # [db], 20log10 (V_out / V_rf)
IIP2 = 40 # [dBm]
IIP3 = 8 # [dBm], using 2 tones at 2.4 GHz and 2.401 GHz

#### Design choices

In [5]:
l = 0.28
gm_id_dp_rf = np.array([12, 12])
gm_id_dp_lo_left = np.array([20, 20])
gm_id_dp_lo_right = np.array([20, 20])

#### Sizing and benchmarking

![IIP3_req](img/IIP3_requirement_formula.png)

In [6]:
jd_rf = n.lookup('ID_W', GM_ID=gm_id_dp_rf, L=l)


In [ ]:
w_rf = id/jd_rf

cgg_w = n.lookup('CGG_W', GM_ID=gm_id_dp_rf, L=l)
cgg = w*cgg_w
ft = gm/cgg/2/np.pi

df = pd.DataFrame([gm_id, id, jd, w, cgg, ft], ['gm_id', 'id', 'jd', 'w', 'cgg', 'ft'], columns=['option1', 'option2', 'option3']); df

#### Spice validation

In [7]:
%%writefile ./sizing_diffpair.spice
** differential pair sizing example

*.lib /foss/pdks/gf180mcuD/libs.tech/ngspice/sm141064.ngspice typical
*.inc /foss/pdks/gf180mcuD/libs.tech/ngspice/design.ngspice

* updated models that allow finger parameterization
.lib ../models_updated_2025.07.19/ngspice/sm141064.ngspice typical
.inc ../models_updated_2025.07.19/ngspice/design.ngspice

.param lx=0.28e-6 wx=146e-6 nfx=30, idx=1e-3

XM1a d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
XM1b d g s 0 nfet_03v3 L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u' pd='2*int((nf+1)/2) * (W/nf + 0.18u)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W' sa=0 sb=0 sd=0 m=1
vg  g  0  1
vd  d  0  2
is  s  0  {2*idx}

.control
  op
  show
  save @m.xM1a.m0[gm]
  save @m.xM1a.m0[cgg]
  save @m.xM1a.m0[cgso]
  save @m.xM1a.m0[cgdo]

  print @m.xM1a.m0[gm]
  print @m.xM1a.m0[cgg]
  print @m.xM1a.m0[cgso]
  print @m.xM1a.m0[cgdo]
    
.endc
.end

Overwriting ./sizing_diffpair.spice


In [8]:
!PDK=gf180mcuD /foss/tools/bin/ngspice -b ./sizing_diffpair.spice

/bin/bash: line 1: /foss/tools/bin/ngspice: No such file or directory
